# Modern Deep Learning: 
# Classify Fashion-MNIST with a simple CNN in Keras

Original version from https://medium.com/tensorflow/hello-deep-learning-fashion-mnist-with-keras-50fcff8cd74a, adapted & extended by Xander Steenbrugge.
![alt text](https://github.com/margaretmz/deep-learning/blob/master/images/modern%20dl_fash-mnist_keras.png?raw=1)

## Why Jupyter Notebook?


*   Interactive programming in the web browser
*   Great for visualization
*   Great for collabration
*   Popular tool for studying machine learning / deep learning




## Why Fashion-MNIST?


*   MNIST is too easy
*   MNIST is overused
*   MNIST can not represent modern Computer Vision tasks

Read more about the Fashion-MINST dataset in this paper [here](https://arxiv.org/abs/1708.07747) (**Fashion-MNIST: a Novel Image Dataset for Benchmarking Machine Learning Algorithms**)



## Notebook Overview

This is a tutorial of how to classify **fashion_mnist** data with a simple **Convolutional Neural Network** in Keras. 
Keras is now part of the core TensorFlow library, in addition to being an independent open source project. 

We will work our way to an efficiently regularized, convolutional network in small, incremental steps, starting from a simple, linear model and gradually adding more and more complexity to our neural network.


The [fashion_mnist](https://github.com/zalandoresearch/fashion-mnist) data: 
60,000 train and 10,000 test data with 10 categories. Each gray-scale image is 28x28.

<br> **Label**	**Description**
<br> 0 T-shirt/top
<br> 1 Trouser
<br> 2 Pullover
<br> 3 Dress
<br> 4 Coat
<br> 5 Sandal
<br> 6 Shirt
<br> 7 Sneaker
<br> 8 Bag
<br> 9 Ankle boot

Each gray-scale image is 28x28

# Setup
First, install TensorFlow version 1.8.0 and import the Fashion-MNIST dataset
We will also be using numpy, matplotlib and the native Keras package, so install those if you haven't already!

Check the requirements.txt file for all necessary dependencies!

In [ ]:
#Test your imports:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import keras

# 1 Data prep

## 1.0 Download the fashion_mnist data
Now, let's download fashion-mnist which is one of the Keras datasets: https://keras.io/datasets/
Pull the dataset from disk into cache memory by storing it in numpy arrays.
As a sanity check, print some shapes to make sure everything is as expected! You should have images of 28 by 28 pixels.

In [ ]:
# Load the fashion-mnist pre-shuffled train data and test data
(x_train_data, y_train_data), (x_test_data, y_test_data) = tf.keras.datasets.fashion_mnist.load_data()
# Define the text labels:
dataset_labels = ["T-shirt/top",  # index 0
                        "Trouser",      # index 1
                        "Pullover",     # index 2 
                        "Dress",        # index 3 
                        "Coat",         # index 4
                        "Sandal",       # index 5
                        "Shirt",        # index 6 
                        "Sneaker",      # index 7 
                        "Bag",          # index 8 
                        "Ankle boot"]   # index 9

#Print some shapes of the data:

## 1.1 Visualize the data
Before we start defining any models, lets have a look at our data to get a sense of what we're dealing with.
Use matplotlib to visualise some of the training images, can you identify the classes yourself?
Perhaps create a simple, callable function that plots a dataset image, given its index.

## 1.2 Data normalization
First investigate the range of the pixel values in the images: what are the min and max you find?

For a neural network to work well, we want these pixel values to be within the range 0-1. Rescale the image pixel values to make this so!

## 1.3 Split the data into train/validation/test data sets
In Machine Learning, we always want to be able to validate how well our model is doing on data it hasn't yet trained on.

*   Training data - used for training the model
*   Validation data - used for tuning the hyperparameters and evaluate the models
*   Test data - used to test the model after the model has gone through initial vetting by the validation set.

Split off 10% of the training data and put this into the validation set!

In [ ]:
validation_fraction = .1

## ToDo ##

(x_train, x_valid) = 
(y_train, y_valid) = 
(x_test, y_test) = 

## 1.4 Some final preprocessing
Most image datasets consist of rgb images. Because of this, Keras will expect each image to have 3 dimensions: [x_pixels, y_pixels, color_channels]. Since our images are grayscale, the color dimension is equal to one and implicitly left out in the numpy arrays. Reshape the image data to contain an explicit color channel of dimension 1. (Your final numpy arrays should have the dimension [nr_samples, x_pixels, y_pixels, 1].)

Secondly, it is very common in deep learning to do classification tasks. To train our model, we will use the categorical crossentropy loss (see https://keras.io/losses/). To compute this, we have to format our image labels as one-hot-vectors. The easiest way to do this is with the tf.keras.utils.to_categorical() function, see: https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical.

In [ ]:
# Reshape input data from (28, 28) to (28, 28, 1)
w, h = 28, 28

#Reshape the data:

#One-hot encode the labels:

#Print some shapes for sanity checking:

# 2 Creating a model

There are two APIs for defining a model in Keras:
1. [Sequential model API](https://keras.io/models/sequential/)
2. [Functional API](https://keras.io/models/model/)

In this notebook we are using the Sequential model API. 
If you are interested in a tutorial using the Functional API, checkout Sara Robinson's blog [Predicting the price of wine with the Keras Functional API and TensorFlow](https://medium.com/tensorflow/predicting-the-price-of-wine-with-the-keras-functional-api-and-tensorflow-a95d1c2c1b03).

In defining the models today, we will be using some of these Keras APIs: (you can check the documentation with these links:)
*   Dense()    [link text](https://keras.io/layers/core/) - Create a fully connected layer
*   Conv2D()   [link text](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D/) - create a convolutional layer 
*   Pooling()  [link text](https://keras.io/layers/pooling/) - create a pooling layer 
*   Dropout()  [link text](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Dropout) - apply drop out 

## 2.0 Baseline: a simple linear model
As a first baseline, we will create a very simple, linear model consisting of one linear transformation matrix and train it with backpropagation.

*  To create the model object, use the tf.keras.Sequential() function as shown here: https://www.tensorflow.org/api_docs/python/tf/keras/models/Sequential.
Since we're not using convolutions for now, we can simply flatten all input pixels into a single row per image containing 28x28 pixel values.

*  Then add a single, linear layer that maps all these pixels onto the 10 output classes. Since the outputs represent class probabilities we can use a softmax activation at the output of our model.

*  You can print a description of your model using model.summary()

Extra question: what does the softmax activation actually do? How does this affect our training procedure?
Here is a good post explaining most of the details: https://medium.com/data-science-bootcamp/understand-the-softmax-function-in-minutes-f3a59641e86d

In [ ]:
# Create the model:
model = 

# Flatten all the pixels (You must define the input shape in the first layer of the neural network)

# Add one Dense layer mapping all those pixels onto 10 output classes (add a softmax activation):

# Take a look at the model summary:
model.summary()

## Compile the model
Configure the learning process with compile() API before training the model. It receives three arguments:

*   An optimizer, we'll use adam
*   A loss function, we'll use the 'categorical_crossentropy'
*   A list of metrics, here we'll start with 'accuracy'

See https://keras.io/models/model/ for details.

In [ ]:
#Compile the model:


## Train the model

Now let's train the model with fit() API.

We use  the [ModelCheckpoint](https://keras.io/callbacks/#modelcheckpoint) API to save the model after every epoch. Set "save_best_only = True" to save only when the validation accuracy improves.

You will also have to configure the batch-size and the number of epochs to train for.
What do these words mean exactly?

In [ ]:
import keras.callbacks
from keras.callbacks import ModelCheckpoint

checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose = 1, save_best_only=True)
model.fit(##ToDo##)

## Load Model with the best validation accuracy
Use the .load_weights() function for this!

In [ ]:
# Load the weights with the best validation accuracy:


## Test Accuracy
Since we used our validation set to choose the best model, we are implicitly already overfitting on this subset of the data. To get the final accuracy estimate our model would get on completely unseen data, compute the accuracy on the test set! Use the .evaluate() function for this.

In [ ]:
# Evaluate the model on test set, and print the accuracy you get:


## Visualize prediction
Now let's visualize the predictions from the model you just trained. 
First we get the predictions with the model from the test data.
Then we visualize some images from the test data set, and set the titles with the prediction (and the groud truth label).
If the prediction matches the true label, the title will be green; otherwise it's displayed in red.
Package everything in a nice, callable function 'visualize_model_predictions(model, x, y)'

In [ ]:
def visualize_model_predictions(model_object, x, y):
    ## ToDo ##
        
visualize_model_predictions(model, x_test, y_test)

## Extra Assignment:
Visualise what this single, linear layer has learned to detect for each class by plotting the weights of trained network.
Our transformation layer is a weight matrix of dimension [28x28, 10]. In a sense, it has learned a 'filter' for each of the 10 classes. Try to plot what these 10 filters look like, by reshaping them into 28x28 images and visualising the results for each class with matplotlib.imshow().

To get the weight matrix of your trained model, you can use the model.layers and .get_weights() functions.

## Extra Assignment 2:
Compare the filters you just plotted with the average image for each class, do you notice any differences? Why?

## Extra assignment 3:
Change the code in block 1.0 to work with MNIST instead of Fashion-MNIST (everything else can stay the same). Rerun all the code you've written so far.

What Test-accuracy do you get with a simple, linear model? Can you see now why MNIST is gradually losing favor as a benchmarking dataset? What about the filter visualisations VS the average images, they look interesting right? Try to think about what exactly is going on here...

# 2.1 Let's turn this into a real neural network!
Add a single hidden layer to the model and add an activation function after that layer (eg 'relu').
The number of hidden units in this layer is usually something in between the input dimension (28x28) and the output dimension (10). Play around with the activation function and the number of hidden units, how do they affect the models performance / training speed?

In [ ]:
model = tf.keras.Sequential()

# Flatten --> Dense_Relu --> Dense_Softmax:



# Take a look at the model summary:
model.summary()

#Compile the model:


In [ ]:
## Add a checkpointer and fit the model on the train data: (you have already done this, see above!)


## Evaluation
Load the best model (according to the validation set) and compute the test accuracy. Re-use your 'visuzalize_model_predictions()' function to display some results.

**Extra question:** Why can't we also visualise the filters learned by this neural network with one hidden layer? What's the problem in this case? Notice how this gives rise to the whole "interpretability" problem in Deep Learning.

In [ ]:
## Load the best model:

## Get test predictions:

## Visualise some results:
visualize_model_predictions(model, x_test, y_test)

## 2.2 Let's make our model Deeper! 
Add two extra hidden layers to start with.
Explore the effects of adding more depth/width to your network, what happens to the number of trainable parameters?
What about training speed, test-accuracy, ... Notice that there usually is a clear tradeoff between training-time and model performance.

In [ ]:
model = tf.keras.Sequential()

# Define the model architecture:

# Take a look at the model summary
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Adding callbacks
Because we just made our model a lot deeper, it will have to train longer in order to use its full potential!
But obviously, we don't want to keep training too long or we'll overfit.
To get more info during training we'll add two extra callbacks to the model.fit() method that:
1. Stop the model training loop if the validation loss keeps dropping consistently (eg 10 times in a row) (This is a default callback, available in Keras)
2. Periodically visualise the validation error while training (This is a custom callback function you will have to create yourself)

You can find more documentation here: https://keras.io/callbacks/ (For the custom callback, scroll down to "Create a callback" section)

In [ ]:
class plot_train_progress(keras.callbacks.Callback):
    def __init__(self):
        #Class attributes eg 'self.plot_frequency'
        # ToDo
        
    def on_train_begin(self, logs={}):
        self.train_losses = []
        self.val_losses = []

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.plot_frequency == 0:
            #plot the training progress:
            #ToDo

    def on_batch_end(self, batch, logs={}):
        self.train_losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

# Early stopping callback:

# Checkpointer callback:

# Plot_train_progress callback:

callbacks_list = [checkpointer, plot_train_progress_cb, earlyStopping]

In [ ]:
model.fit(x_train,
         y_train,
         batch_size=128,
         epochs=25,
         validation_data=(x_valid, y_valid),
         callbacks=callbacks_list)

## Evaluate the model

In [ ]:
model.load_weights('model.weights.best.hdf5')
score = model.evaluate(x_test, y_test, verbose=0)

print('Test accuracy: %.2f%%' %(100*score[1]))

**Extra:** To experiment and get some intuition as to why making the network deeper makes it easier to fit the data, you can experiment with this amazing in-browser tool: https://playground.tensorflow.org/

## 2.2 Convolutions 
### Lets add in some of the big guns in Computer Vision: Convolutions!

Notice that up until now, we've always started our network with a Flatten() operation that basically throws away all the spatial dependency information in our image. Time to change that!

Chain two Conv2D layers in the beginning of the network (remove the Flatten operation) and follow each one with a MaxPooling2D layer to downscale the resolution of the hidden representations. Finally add some normal FC layers to map the convolutional features to the output classes. (You will need to re-add a Flatten() operation for this).
Play around with different architectures and keep track of the following variables:
- Total number of trainable parameters in the network
- Network depth (how many non-linear layers do I have that transform the data from pixel-space to label-space?)
- Training time per epoch
- Final accuracy after training for N epochs

You can find the documentation for the Conv2D layers here: https://keras.io/layers/convolutional/

**Extra question:** What does the MaxPooling2D operation do exactly? Do we lose any information in this step? What is good/bad about that?

In [ ]:
model = tf.keras.Sequential()

# Define the model:

# Take a look at the model summary
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#Define the callbacks you want:

callbacks_list = # ToDo

model.fit(x_train,
         y_train,
         batch_size=128,
         epochs=25,
         validation_data=(x_valid, y_valid),
         callbacks=callbacks_list)

## Evaluate model

In [ ]:
model.load_weights('model.weights.best.hdf5')
score = model.evaluate(x_test, y_test, verbose=0)

print('Test accuracy: %.2f%%' %(100*score[1]))

**Extra**:
Run the visualize_model_predictions() function both on some examples from the training data and the test data, do you notice any differences? Is there an overall difference between the accuracy on the Train-set vs the Test-set? What is going on?

In [ ]:
visualize_model_predictions(model, x_test, y_test)
visualize_model_predictions(model, x_train, y_train)

#Compute & compare train and test accuracies:


# 3 Regularization

Our model currently has a lot of degrees of freedom (it has a LOT of trainable parameters and can therefore fit almost any function if we just keep training for long enough). This means our network is also prone to overfitting. 

In this section, let's add dropout layers between the main parts of our network to avoid overfitting.
What is a good dropout rate?

**Extra:** To get an idea of the degree to which these networks can overfit take a look at this amazing paper which shows that a large enough CNN can basically memorize a very large, random dataset: https://arxiv.org/abs/1611.03530

**Extra:** Also try to add an L2 Regularization penalty to every layer in your network. What is a good setting of the multiplier constant? (see https://keras.io/regularizers/)

In [ ]:
model = tf.keras.Sequential()

# Define the model:

# Take a look at the model summary
model.summary()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
callbacks_list = # ToDo

model.fit(x_train,
         y_train,
         batch_size=128,
         epochs=25,
         validation_data=(x_valid, y_valid),
         callbacks=callbacks_list)

## Evaluate model:

In [ ]:
model.load_weights('model.weights.best.hdf5')

test_score = model.evaluate(x_test, y_test, verbose=0)
train_score = model.evaluate(x_train, y_train, verbose=0)
print('Train accuracy: %.2f%% || Test accuracy: %.2f%%' %(100*train_score[1], 100*test_score[1]))

# 4 Data Augmentation
In the real world, getting enough training data is often a problem. To show the effect of this, training your architecture from above using only 100 training images, what is the best test-accuracy you can get now?

In [ ]:
model = tf.keras.Sequential()

# You can use the previous model architecture:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
nr_samples_to_use = 100
batch_size = 32

callbacks_list = # Whatever you want to use

model.fit(x_train[:nr_samples_to_use],
         y_train[:nr_samples_to_use],
         batch_size=batch_size,
         epochs=50,
         validation_data=(x_valid, y_valid),
         callbacks=callbacks_list)

In [ ]:
model.load_weights('model.weights.best.hdf5')
score = model.evaluate(x_test, y_test, verbose=0)

print('Test accuracy: %.2f%%' %(100*score[1]))

## Lets augment our small Training-dataset at runtime using an ImageDataGenerator():
See https://keras.io/preprocessing/image/

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# this is the augmentation configuration we will use for training:
train_datagen = ImageDataGenerator( #ToDo )

# this is the augmentation configuration we will use for testing:
validation_datagen = ImageDataGenerator()
test_datagen = ImageDataGenerator()

# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
train_datagen.fit(x_train)
validation_datagen.fit(x_valid)
test_datagen.fit(x_test)

## Plot a few batches of augmented images to check the result of your Data Augmentation parameters
Make sure you can still classify every augmented image yourself!

## Use the ImageDataGenerator to augment the small Train-subset at runtime.
What test-accuracy do you get now?

In [ ]:
model = tf.keras.Sequential()

# Use the same model as before

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
callbacks_list = # Up to you

# fits the model on batches with real-time data augmentation:
model.fit_generator(generator = train_datagen.flow(x_train[:nr_samples_to_use], y_train[:nr_samples_to_use], batch_size=batch_size),
                    validation_data = validation_datagen.flow(x_valid, y_valid, batch_size=batch_size), 
                    steps_per_epoch = nr_samples_to_use // batch_size,
                    callbacks=callbacks_list,
                    use_multiprocessing=True,
                    workers=8,
                    epochs=100)

In [ ]:
model.load_weights('model.weights.best.hdf5')
score = model.evaluate(x_test, y_test, verbose=0)

print('Test accuracy: %.2f%%' %(100*score[1]))

In [ ]:
## Note: Data Augmentation on Fashion-MNIST is not the most effective. 
## But the concept is very generally usefull for a lot of Deep Learning problems, hence the inclusion here!

# Final assignment
### Finally, throw everything together and try to get the best possible performance on the Test-set! Who will win?

For the Pro's, here are some additional tips:
* Try Batch-Normalization layers
* Play around with the Data Augmentation parameters a lot: augmenting data is good, but you don't want to push your training data distribution to far out (the test-data is not augmented!!) For example: sneakers, sandals and boots are always facing left in the Fashion-MNIST dataset, so random-horizontal flipping isn't the best option here...
* You could also experiment with some of the newer activation functions like LeakyReLU, SeLU, ...
* Cyclical learning rates
* ...